In [10]:
import numpy as np
import mplhep as hep
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, accuracy_score
import uproot as up
import pandas as pd 
import awkward as ak
import os

os.environ["HOMELUDO"] = '/home/submit/mori25/'
os.environ["DPLUDO"] = '/work/submit/mori25/Darkphotons_ludo/'

from training_offline import Trainer, plot_ROC

In [11]:
Y_forest = Trainer("Y", 'forest_standard')
Y_forest.complete_load()
Y_forest.plot_model()

Successfully imported data file to memory


/work/submit/mori25/Darkphotons_ludo/BDT/training_offline.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sig['Score'] = 1
/work/submit/mori25/Darkphotons_ludo/BDT/training_offline.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bkg['Score'] = 0


Defined training and evaluation datasets
[18:56:16] /home/conda/feedstock_root/build_artifacts/xgboost-split_1691196879049/work/src/common/io.cc:102: Opening /work/submit/mori25/Darkphotons_ludo/BDT/trained_models_Y/forest_standard.json failed: No such file or directory
Stack trace:
  [bt] (0) /work/submit/mori25/anaconda3/envs/rhino/lib/libxgboost.so(+0x20b7df) [0x7efc6d0597df]
  [bt] (1) /work/submit/mori25/anaconda3/envs/rhino/lib/libxgboost.so(xgboost::common::LoadSequentialFile(std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> >, bool)+0x8f4) [0x7efc6d0aed24]
  [bt] (2) /work/submit/mori25/anaconda3/envs/rhino/lib/libxgboost.so(+0x22d441) [0x7efc6d07b441]
  [bt] (3) /work/submit/mori25/anaconda3/envs/rhino/lib/libxgboost.so(XGBoosterLoadModel+0x121) [0x7efc6d07b811]
  [bt] (4) /work/submit/mori25/anaconda3/envs/rhino/lib/python3.11/lib-dynload/../../libffi.so.8(+0x6a4a) [0x7efd35906a4a]
  [bt] (5) /work/submit/mori25/anaconda3/envs/rhino/lib/python3.11/

XGBoostError: [18:56:16] /home/conda/feedstock_root/build_artifacts/xgboost-split_1691196879049/work/src/learner.cc:792: Check failed: mparam_.num_feature != 0 (0 vs. 0) : 0 feature is supplied.  Are you using raw Booster interface?
Stack trace:
  [bt] (0) /work/submit/mori25/anaconda3/envs/rhino/lib/libxgboost.so(+0x20b7df) [0x7efc6d0597df]
  [bt] (1) /work/submit/mori25/anaconda3/envs/rhino/lib/libxgboost.so(+0x3e04ee) [0x7efc6d22e4ee]
  [bt] (2) /work/submit/mori25/anaconda3/envs/rhino/lib/libxgboost.so(+0x3e7453) [0x7efc6d235453]
  [bt] (3) /work/submit/mori25/anaconda3/envs/rhino/lib/libxgboost.so(+0x3e0b1a) [0x7efc6d22eb1a]
  [bt] (4) /work/submit/mori25/anaconda3/envs/rhino/lib/libxgboost.so(XGBoosterPredictFromDMatrix+0x2f9) [0x7efc6d079639]
  [bt] (5) /work/submit/mori25/anaconda3/envs/rhino/lib/python3.11/lib-dynload/../../libffi.so.8(+0x6a4a) [0x7efd35906a4a]
  [bt] (6) /work/submit/mori25/anaconda3/envs/rhino/lib/python3.11/lib-dynload/../../libffi.so.8(+0x5fea) [0x7efd3590

In [ ]:
Y_tree = Trainer("Y", 'tree_standard')
Y_tree.complete_load()
Y_tree.plot_model()

In [ ]:
plot_ROC(Y_tree.dtrain,Y_tree.dval,[Y_tree.bst,Y_forest.bst], ["GB tree", "Boosted RF"])

Now look for optimal discriminators

In [ ]:
predY = Y_tree.bst.predict(xgb.DMatrix(Y_tree.trainData_skinny))

In [ ]:
#Study cuts on discriminators to optimize for Y

id1Y=Y_tree.trainData["Muon_softMva1"]
id2Y=Y_tree.trainData["Muon_softMva2"]
sig_cut = Y_tree.trainData["Score"]==1
bkg_cut = Y_tree.trainData["Score"]==0

def efficiencyY(id_lim,dis_lim):
    sig = (predY[sig_cut] > dis_lim) & (id1Y[sig_cut] > id_lim) & (id2Y[sig_cut] > id_lim)
    s = np.sum(sig)
    bkg = (predY[bkg_cut] > dis_lim) & (id1Y[bkg_cut] > id_lim) & (id2Y[bkg_cut] > id_lim)
    b = np.sum(bkg)
    return s/np.sqrt(b) if b!= 0 else 0


In [ ]:
id_vals = np.linspace(0.2,0.5,50)
vtx_vals= np.linspace(0.45,0.9,50)
XY, YY = np.meshgrid(id_vals,vtx_vals)
effY = np.vectorize(efficiencyY)(XY,YY)
idxY = np.unravel_index(np.argmax(effY),effY.shape)
print("Optimal discriminator cuts: \n SoftMVA (Muon ID): ", XY[idxY], "\n BDT (Vertex quantities): ", YY[idxY])

In [ ]:
plt.figure(figsize=(10, 9))
pcm = plt.pcolormesh(XY, YY, effY, shading='auto', cmap='inferno')# norm=matplotlib.colors.LogNorm()
colorbar = plt.colorbar(pcm, label='$s/\sqrt{b}$')
# colorbar_ticks = [1000,1300 , 2000]  # Define your desired tick positions
# colorbar.set_ticks(colorbar_ticks)
plt.xlabel('MuonID MVA cut')
plt.ylabel('BDT cut')
plt.title('Discriminator yield for Y peak')
# plt.ylim(0.65,0.85)
plt.show()